In [1]:
from threading import Thread
import json
from stmpy import Machine, Driver
import paho.mqtt.client as mqtt
from utils import BROKER, PORT, TOPIC, JOIN_TOPIC, QUEUE_TOPIC, HELP_TOPIC, UPDATE_TOPIC 


# TODO: fikse casen hvor man starter lab session ved exit fra wait state når du får feil kode

class MQTT_Client_1:
    def __init__(self, stm):
        self.state_machine = stm
        self.count = 0
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.stm_driver: Driver = None

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        try:
            #message = json.loads(msg.payload.decode('utf-8'))
            message: dict = json.loads(msg.payload.decode('utf-8'))
        except json.decoder.JSONDecodeError:
            return
    
   
        #self.state_machine.message = message
        print(message)
        print("on_message(): topic: {}".format(msg.topic))
        
        if msg.topic == JOIN_TOPIC:
            print("Riktig sted")
            print(message['msg'])
            if message['msg'] == "session_joined":
                print("Correct code")
                self.stm_driver.send("correct_code", "student")

                #Unsubscribe from the JOIN TOPIC so messages are received

                self.client.unsubscribe(JOIN_TOPIC)

                # Send the session_id to the student object
                self.state_machine.session_id = message["session_id"]
                
                #Subscribe to the session and the additional topics
                
                
                self.client.subscribe(f"{TOPIC}/{message['session_id']}/{HELP_TOPIC}")
                print("Sucsessfully subscribed to: "+ (f"{TOPIC}/{message['session_id']}/{HELP_TOPIC}"))
                
                self.client.subscribe(f"{TOPIC}/{message['session_id']}/{UPDATE_TOPIC}")
                print("Sucsessfully subscribed to: "+ (f"{TOPIC}/{message['session_id']}/{UPDATE_TOPIC}"))
                
                                      

        elif msg.topic == JOIN_TOPIC:
            if message == "Invalid code":
                print("Feil kode")
                self.stm_driver.send("wrong_code", "student")
        
        """if msg.topic == QUEUE_
             self.state_machine.message = message["queue"]            """



        

    
        
    def start(self, broker, port):
        #Kobler til main topic og join topic og broker
        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)
        self.client.subscribe(TOPIC)
        self.client.subscribe(JOIN_TOPIC)

        try:
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()
            
            


In [2]:
!pip3 install paho-mqtt
from stmpy import Driver, Machine
from threading import Thread
import paho.mqtt.client as mqtt
import json
from stmpy import Machine, Driver
import ipywidgets as widgets
from IPython.display import display
import random
import sys

#NOTES:
## Håndtere i server som mottar join-koder at den enten sender trigger: riktig kode, eller trigger: feil kode, 
# utifra det så vet vi hvilken state vi skal tils
#Hver eneste student må ha en slags kode/gruppenavn


class Student:
      def __init__(self):
            self.button_create = widgets.Button(description="Join Session")
            self.button_create.on_click(self.on_button_join)
            self.message = ""
            self.session_id = ""
            self.mqtt_client: mqtt.Client = None
            self.stm: Machine = None
            self.question = 1
            # MIDLERTIDIG
            self.total_questions = 5
            # text field
            self.stu_code = widgets.Text(value='', placeholder='', description='student code:', disabled=False)
            print(self.stu_code.value)
            display(self.button_create, self.stu_code)
            
            self.group_name = widgets.Text(value='', placeholder='', description='Group name:', disabled=False)
            display(self.button_create, self.group_name)
            display()

      def on_wait(self):
            """TODO"""


      
      def ask_help(self):

            question_for_server_use = "self.group_name +  + self.question"

            dictMessage = { "question": self.question, "group_name": self.group_name.value}
            print(self.session_id)
            jsonMessage = json.dumps(dictMessage)
            self.mqtt_client.subscribe(f"{TOPIC}/{self.session_id}/{QUEUE_TOPIC}")
            self.mqtt_client.publish(f"{TOPIC}/{self.session_id}/{HELP_TOPIC}", jsonMessage)
            
            print("Sucsessfully subscribed to: "+ (f"{TOPIC}/{self.session_id}/{QUEUE_TOPIC}"))
            #DISABLE HELP BUTTON)

      def finish_task(self):
            pasram = "self.group_name + self.question"
            dictMessage = {  "question": self.question, "group_name": self.group_name.value}
            jsonMessage = json.dumps(dictMessage)
           # self.mqtt_client.publish(f"{TOPIC}/{message['session_id']}/{UPDATE_TOPIC}", jsonMessage)
            self.question += 1
            self.progress_bar.value += 1    

      def update_queue(self):
            self.place = 1
            if self.group_name == self.message:
                  self.mqtt_client.unsubscribe(QUEUE_TOPIC)
            #ENABLE HELP
            for i in self.mesage:
                  if i == self.group_name:
                        break
                  else:
                        self.place += 1
    

      def publish(self):
            print(self.stu_code)
            code = self.stu_code.value
            group_name = self.group_name.value

            dictMessage = {"msg": "join_session"," code": code, "group_name": group_name}
            jsonMessage = json.dumps(dictMessage)
            print(jsonMessage)

            self.mqtt_client.publish(JOIN_TOPIC ,jsonMessage)
            self.stu_code.value = "Loading"
      
      def setup_lab(self):
            self.button_help = widgets.Button(description="Get help")
            self.button_help.on_click(self.on_button_ask_help)
            display(self.button_help)
            self.button_finish_task = widgets.Button(description="Finish task")
            self.button_finish_task.on_click(self.on_button_finish_task)
            display(self.button_finish_task)
            self.progress_bar = widgets.IntProgress(min = 0, max = self.total_questions, description = "Progress")
            #self.progress_bar.value(self.question)
            display(self.progress_bar)
            pass

      def save_text(self):
           self.code = self.stu_code
           

      def on_button_join(self, session_code_stu):
        self.stm.send('join') # <---- here we send a message
        print(self.stm.state)
        pass
  
      def on_button_finish_task(self, session_code_stu):
            self.stm.send('finish_task')
            pass
      
      def on_button_ask_help(self, session_code_stu):
            self.stm.send('ask_help')
            pass
# initial transition
t0 = {'source': 'initial',
      'target': 'idle'}

# transitions

#t1 and t2 are transitions from idle to wait and back
t1 = {'trigger':'join', 
      'source':'idle', 
      'target':'wait'}


t2 = {'trigger':'wrong_code', 
      'source':'wait', 
      'target':'idle'}


t3 = {'trigger':'correct_code', 
      'source':'wait', 
      'target':'lab_session_active',
      'effect': 'setup_lab'}


t4 = {'trigger':'ask_help', 
      'source':'lab_session_active', 
      'target':'lab_session_active',
      'effect': 'ask_help' }

t5 = {'trigger':'leave_queue', 
      'source':'lab_session_active', 
      'target':'lab_session_active',
      'effect': 'leave_queue'}

t6 = {'trigger':'finish_task', 
      'source':'lab_session_active', 
      'target':'lab_session_active',
      'effect': 'finish_task'}

#Trengs ikke siden det skjer automatisk når man ber om hjelp
t7 = {'trigger':'queue_request', 
       'source':'lab_session_active', 
       'target':'lab_session_active',
       'effect': 'update_queue'
       }


t8 = {'trigger':'leave_lab', 
      'source':'lab_session_active', 
      'target':'exit'}




# the states:
idle = {'name': 'idle'}

wait = {'name': 'wait',
        'entry': 'publish'}
#TODO move setup_lab to right place

lab_session_active = {'name': 'lab_session_active'}

    

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [ ]:
student = Student()
state_machine = Machine(transitions=[t0, t1, t2, t3, t4, t5, t6, t7, t8], states=[idle, wait, lab_session_active], obj=student, name="student")
student.stm = state_machine

driver = Driver()
driver.add_machine(state_machine)

myclient = MQTT_Client_1(student)
student.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(BROKER, PORT)

Button(description='Join Session', style=ButtonStyle())

Text(value='', description='student code:', placeholder='')

Button(description='Join Session', style=ButtonStyle())

Text(value='', description='Group name:', placeholder='')

Connecting to mqtt20.iik.ntnu.no:1883


on_connect(): Connection Accepted.
idle
Text(value='muslimer', description='student code:', placeholder='')
{"msg": "join_session", " code": "muslimer", "group_name": "brenn de og"}
{'msg': 'join_session', ' code': 'muslimer', 'group_name': 'brenn de og'}
on_message(): topic: join
Riktig sted
join_session
{'msg': 'session_joined', 'session_id': '69420', 'group_name': 'pikk'}
on_message(): topic: join
Riktig sted
session_joined
Correct code
Sucsessfully subscribed to: team02/69420/help
Sucsessfully subscribed to: team02/69420/update


Button(description='Get help', style=ButtonStyle())

Button(description='Finish task', style=ButtonStyle())

IntProgress(value=0, description='Progress', max=5)

69420
Sucsessfully subscribed to: team02/69420/queue
{'question': 1, 'group_name': 'brenn de og'}
on_message(): topic: team02/69420/queue
